In [ ]:
import os
import subprocess

# Base directory where minim folders are located
base_scr_dir = "/home/hp/results/humansec"

# Directories for input files
run_dir = "/home/hp/nayanika/github/GPX6/cluster_scripts"
genfep_dir = "/home/hp/nayanika/github/GPX6/input"
topology_dir = "/home/hp/nayanika/github/GPX6/prep_structures/humansec"
fep_dir = "/home/hp/nayanika/github/GPX6/input/fep"

# Iterate over each system folder within the base directory
for system_name in os.listdir(base_scr_dir):
    system_dir = os.path.join(base_scr_dir, system_name)
    minim_dir = os.path.join(system_dir, "minim")
    
    # Ensure the minim directory exists
    if not os.path.isdir(minim_dir):
        print(f"Skipping {system_dir} as minim directory does not exist.")
        continue

    # Define the paths for the required files
    pdb_file = os.path.join(topology_dir, f"{system_name}.pdb")
    topology_file = os.path.join(topology_dir, f"{system_name}.top")
    relax_input_path = os.path.join(minim_dir, "relax_012.inp")
    
    # Check if the required files exist
    if not os.path.isfile(pdb_file):
        print(f"No .pdb file found for {system_name} at {pdb_file}")
        continue
    if not os.path.isfile(topology_file):
        print(f"No topology file found for {system_name} at {topology_file}")
        continue
    if not os.path.isfile(relax_input_path):
        print(f"Error: Relax input file '{relax_input_path}' doesn't exist for {system_name}.")
        continue

    # Construct the command to run q_genfeps.py
    command = [
        "q_genfeps.py",
        os.path.join(genfep_dir, "selgenfeps.proc"),
        "--pdb", pdb_file,
        relax_input_path,  # Use the system-specific relax input file path
        "relax",
        "--repeats", "100",
        "--frames", "51",
        "--fromlambda", "1.0",
        "--prefix", os.path.join(system_dir, "replica"),
        "--rs", os.path.join(run_dir, "run_qdyn_5.sh")
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
        print(f"Successfully processed {system_name} with q_genfeps.py.")
    except subprocess.CalledProcessError as e:
        print(f"Error processing {system_name}: {e}")

print("Processing completed for all systems.")
